In [ ]:
#!pip install --upgrade beautifulsoup4

In [ ]:
import requests
import pandas as pd
from datetime import datetime
from datetime import date
import time
import pickle
from bs4 import BeautifulSoup

## 1. Build port 

In [ ]:
response = requests.get(
    url='https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States'
)
print(response.status_code) # should be 200

## 2. Table Scrapping

In [ ]:
response = requests.get(
      url='https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States'
#     url =r'https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States_(before_2000)'
)
print(response.status_code) # should be 200

soup = BeautifulSoup(response.text,'lxml')

# extract pure data 
data = []
colum_names = []
# find all sortabel tables
for i in soup.find_all(name='table'):
    
    # find all column names 
    for j in i.find_all(name='th'):
        colum_names.append(j.text.rstrip())
    
    # find each row
    for k in i.find_all(name='tr'):
        
        each_row ={}
        # each item in the row
        for index,item in enumerate(k.find_all(name='td')):
            each_row[colum_names[index]] = item.text.rstrip()
        
        data.append(each_row)
        
df = pd.DataFrame(data)

df = df.dropna(axis=0).reset_index(drop=True) 

df.head()

## 3. Scrapping Latitude and Longitude of citys

In [ ]:
data = []
loc_href = []
for i in soup.find_all(name='table',attrs={'class':'sortable wikitable'}):
    for j in i.find_all(name='tr'):
        
        each_row ={}
        iter_ = iter(j.find_all('td'))
        while True:
            try:
                each_row['Date']=next(iter_).text.rstrip()
                loc = next(iter_)
                each_row['Location']=loc.text.rstrip()
                # find latitude and logitude
                re_loc = requests.get(
                    url='https://en.wikipedia.org{}'.format(loc.a['href'])
                )

                bs = BeautifulSoup(re_loc.text,'lxml')
                b = bs.find('span',attrs={'class':'plainlinks nourlexpansion'})
                each_row['cordinate'] = (b.find(name='a',attrs={"class":"external text", "rel":"nofollow"}).find(name='span',attrs={'class':'geo-dec'}).text)
                # end
                
                
                #loc_href.append(loc.a['href'])
                each_row['Deaths']=next(iter_).text.rstrip()
                each_row['Injuries']=next(iter_).text.rstrip()
                each_row['Description']=next(iter_).text.rstrip()
            except StopIteration: 
                break
            
        data.append(each_row)
        #print('\n')
        

In [ ]:
with open('raw_data.pkl','wb') as f:
    pickle.dump(data,f)

## 4. Data Clean 

In [ ]:
df = pd.DataFrame(data[1:])

df = df.dropna(axis=0).reset_index(drop=True)

df[['City','State']]=df.Location.str.split(',',expand=True)
df.drop(columns=['Location'],inplace=True)

df['Deaths'] = df.Deaths.str.extract(r'(\d+)')
df['Injuries'] = df.Injuries.str.extract(r'(\d+)')


df[['Latitude','Longitude']] = df.cordinate.str.split(' ',expand=True)
df.drop(columns=['cordinate'],inplace =True)


df['Latitude'] = df.Latitude.str.extract(r'(\d+\.\d+)')
df['Longitude'] = df.Longitude.str.extract(r'(\d+\.\d+)')

df['Longitude'] = df['Longitude'].apply(lambda x: '-'+x)

df['Longitude']= df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)

df['Date'] = df['Date'].apply(lambda x:datetime.strptime(x,'%B %d, %Y'))
df['Description'] =df['Description'].str.replace(r'(\[\d+\])*','')

In [ ]:
df.isna().any()

In [ ]:
with open('processed_data.pkl','wb') as f:
    pickle.dump(df,f)